# Aperture Photometry Pipeline

<b>Description:</b> This pipeline is meant to be part of a three-step process for performing timing analysis on observations of a target time-variable object. The first step of the process was to reduce FITS files of the object by debiasing, dark-subtracting, and flat-fielding. For the pipeline corresponding to the first step, refer to <i>image_reduction_pipeline.ipynb</i>. The second step of the process is to perform aperture photometry on the reduced FITS files and extract the apparent magnitude(s) for the desired object(s). For the pipeline corresponding to the second step, refer to <i>aperture_photometry_pipeline.ipynb</i> (this file). The third step of this process is to perform the timing analysis on the extracted magnitudes themselves. For the pipeline corresponding to the third step, refer to <i>timing_analysis_pipeline.ipynb</i>.

<b>This Jupyter Notebook file will perform the second step of this process: aperture photometry of the reduced FITS files</b>. It will take in the reduced FITS files and extract the apparent magnitudes of the object of observation as well as of any additional comparison stars (in our case, 5 comparison stars). 

<b>Directions:</b> You will need to manually feed in the initial x and y pixel coordinates of the object and comparison stars in the pipeline for the first image, but then the pipeline will automatically be able to find the x and y pixel coordinates of the objects for future frames. If the objects move a lot between frames, you have the option of specifying additional initial x and y coordinates that the pipeline can use to search for the object. <b>Note that the order in which in you input your list of initial coordinates matters</b>: the pipeline will use the first list of initial coordinates to search for your objects, and only when it fails to find the object will it move onto the next list of initial coordinates. There is no limit to the number of initial coordinates you can put into the pipeline. You will also need to manually input the directories in which the reduced FITS files are located. The pipeline will automatically subtract out the sky when performing aperture photometry, so you do not need to worry about that step, but it will NOT automatically debias, dark-subtract, or flat-field the images, so make sure you have done those three steps beforehand.
<br>Sometimes the pipeline will still not find your object in spite of the initial coordinates you gave it. In this case, you may have to manually rimexam the object.

<b>Output:</b> The output of the pipeline are a collection of text files, one for each directory that you fed into the pipeline. Each text file contains the x and y centroids of the objects, their aperture sum, their apparent magnitudes, the radius used for the aperture, the background sky value, the name of the object, and the FITS filename in which the flux was taken.

## How to Run This File

### The command line arguments to run this Python file is written below:
#### python aperture_photometry_pipeline.py dir_proc_fits coord_file.txt offset=10 radius=5
<ul>
    <li>The first argument is the name of this python file (aperture_photometry_pipeline.py).
    <li>The second argument is the PATH of the directory containing all your reduced fits files.
    <li>The third argument is the name of the text file containing the initial coordinates of your object and comparison stars. This text file should be formatted as follows:
<br><i>name1:(x,y)</i>
<br><i>name2:(x,y)</i>
<br><i>name3:(x,y)</i>
        
For example, if you think the object won't move too much in between frames, then the text file will look something like this:
<br>A0620:(478, 565)
<br>compstar1:(385, 535)
<br>compstar2:(381, 475)
<br>compstar3:(733,713)
<br>compstar4:(557,286)
<br>compstar5:(789,293)
        
But if you think the object WILL move a lot in between frames, then specify the second list of initial coordinates below the first, like this:
<br>A0620:(478, 565)
<br>compstar1:(385, 535)
<br>compstar2:(381, 475)
<br>compstar3:(733,713)
<br>compstar4:(557,286)
<br>compstar5:(789,293)
<br>A0620:(498, 585)
<br>compstar1:(405, 555)
<br>compstar2:(401, 495)
<br>compstar3:(753,733)
<br>compstar4:(577,306)
<br>compstar5:(809,313)
<br>There is no limit to the number of initial coordinates you can put into the text file.
    <li>The (optional) fourth argument is the maximum pixel movement of the objects between frames due to movement of the night sky and/or proper motion of the object that the pipeline should account for when searching for the object between frames. This is automatically set to 10, since this yielded the best results for previous A0620-00 data. 
<br><i>Note: a higher offset will decrease the accuracy of the pipeline finding the correct centroid of your objects as you move from frame to frame, since the pipeline searches in a wider area around the previous frame's centroid coordinates.</i>
    <li>The (optional) fifth argument is the radius of the circular aperture to use when summing the flux around each object. This is automatically set to 5 since that's the previous radius we used in our existing A0620-00 data.
</ul>

### Assumptions Implicit in This Pipeline
This pipeline assumes that the object and comparison stars don't overlap in the night sky with any other point sources, and that the fits files fed into the pipeline are already reduced.

## Importing Packages

In [1]:
# Importing packages
import numpy as np
import os
import sys
from astropy.io import fits
from astropy.table import vstack, QTable
from astropy.time import Time
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
import matplotlib.pyplot as plt

## Reading in Command Line Arguments

In [2]:
# Just for testing purposes
#sys.argv[1] = "test_dirs_Dec\I-bess"
#sys.argv[2] = "test_dirs_Dec\I-bess\Dec_coords_I.txt"

Defining a helper function that returns whether a file's extension is '.fits' or not.

In [3]:
def has_fits_file_extension(filename):
    '''A function that returns whether a file's extension is '.fits' or not.'''
    return filename.split(".")[-1] == "fits"

Reading in the directory for the reduced fits files and the coordinate text file, and (optionally) the offset and radius.

In [4]:
if len(sys.argv) == 1:
    raise TypeError("aperture_photometry_pipeline.py missing two positional arguments: 'proc_files_dir', and 'coordinates.txt'")
if len(sys.argv) == 2:
    raise TypeError("aperture_photometry_pipeline.py missing one positional argument: 'coordinates.txt'")

fits_path = sys.argv[1]
fits_filenames = list(filter(has_fits_file_extension, os.listdir(fits_path)))
coord_file = sys.argv[2]
names = []
starting_positions = []
with open(coord_file, "r+") as file:
    text = file.read()
    lines = text.split('\n')
    for line in lines:
        name, coord = line.split(':')[0], line.split(':')[1]
        xcor, ycor = int(float(coord.split(',')[0][1:])), int(float(coord.split(',')[1][:-1]))
        if name in names:
            starting_positions[names.index(name)].append([xcor, ycor])
        else:
            names.append(name)
            starting_positions.append([[xcor, ycor]])
starting_positions = np.array(starting_positions)

offset = sys.argv[3] if len(sys.argv) > 3 else 10
radius = sys.argv[4] if len(sys.argv) > 4 else 5

## Obtaining the Apparent Magnitudes

Defining helper functions.

In [46]:
def centroid(arr):
    x_sum = 0
    y_sum = 0
    for i in range(len(arr)):
        for j in range(len(arr[i])):
            x_sum += arr[i][j] * j
            y_sum += arr[i][j] * i
    x_com = x_sum / np.sum(arr)
    y_com = y_sum / np.sum(arr)
    return x_com, y_com

def find_objects_stats(data, positions, file):
    centroids = []
    peaks = []
    for i in range(len(positions)):
        if positions[i][1] - offset < 0:
            positions[i][1] = offset
            print(positions[i][1])
        elif positions[i][1] + offset >= len(data):
            positions[i][1] = len(data) - offset - 1
        elif positions[i][0] - offset < 0:
            positions[i][0] = 0
        elif positions[i][0] + offset >= len(data[0]):
            positions[i][0] = len(data[0]) - offset - 1
            
        data_region = data[positions[i][1] - offset : positions[i][1] + offset, positions[i][0] - offset : positions[i][0] + offset] # Defining the area in which to look for object
        xsub, ysub = centroid(data_region) # Finding centroid, but xsub and ysub have wrong index, so need to reindex
        cen = (xsub + positions[i][0] - offset, ysub + positions[i][1] - offset) # reindexing
        peak = data[int(cen[1])][int(cen[0])]
        centroids.append(cen)
        peaks.append(peak)
    
    # Finding the aperture sum of a circular region around each object and putting it into a QTable
    aperture = CircularAperture(centroids, r=radius)
    phot_table = aperture_photometry(data, aperture)
    phot_table["xcenter"] = phot_table["xcenter"].value # Removing the unit quantity in xcenter and ycenter
    phot_table["ycenter"] = phot_table["ycenter"].value
    return phot_table, peaks

Obtains the apparent magnitudes of the object(s) and comparison stars, and then outputs them into an astropy QTable.

In [49]:
# Creating a dummy table
aperture_table = QTable({'xcenter': [], 'ycenter': [], 'aperture_sum': [], 'aperture_mag': [], 'R': [], 'sky': []})

# Setting the initial position to search for the object(s) when finding centroids
positions = starting_positions[:,0] # Gets the first element in every row
starting_positions_index = 0

# Routine to obtain the apparent magnitudes
for file in fits_filenames:
    # Opening the fits file
    hdulist = fits.open(os.path.join(fits_path, file))
    data_wsky = hdulist[0].data

    # Subtracting out global sky background
    skybkg = np.median(data_wsky)
    data = data_wsky - skybkg
    
    # Finding centroid positions of A0620-00 and comparison stars using photutils.centroids package
    phot_table, peaks = find_objects_stats(data, positions, file)
    
    # If aperture sum is negative, it most likely means the centroid has not been correctly found, most likely due to
    #   extreme movement of the CCD camera and/or object in night sky, causing the coordinates of the object to shift
    #   beyond the range of 'offset'. In this case, if we have another list of initial starting coordinates
    #   in which to look for our objects, restart the centroid search with these coordinates; otherwise, continue as normal.
    if any(i < 0 for i in phot_table["aperture_sum"]):
        starting_positions_index += 1
        if starting_positions_index < len(starting_positions[0]):
            positions = starting_positions[:,starting_positions_index]
            phot_table, peaks = find_objects_stats(data, positions, file)
    
    # Finding the corresponding apparent magnitude and adding as a new column in the table. The flux and magnitude zeropoints were determined using curve_fits to previous A0620-00 data.
    fluxzero = 10.91926955
    magzero = 22.40441725
    mag = -2.5*(np.log10(phot_table['aperture_sum'] / fluxzero)) + magzero
    phot_table["aperture_mag"] = mag
    
    # Finding the peak counts and adding it as a new column in the table.
    phot_table["peak"] = peaks
    
    # Adding name of the object, radius, background sky brightness, MJDs (modified Julian dates), exposure times, and filename as extra columns for future reference
    phot_table['name'] = names
    phot_table['R'] = [radius] * len(positions)
    phot_table["sky"] = [skybkg] * len(positions)
    
    UTCSTART = hdulist[0].header["UTCSTART"]
    MJD = Time(UTCSTART, format="isot", scale="utc").mjd
    phot_table["MJD"] = [MJD] * len(positions)
    
    exptime = hdulist[0].header["EXPTIME"]
    phot_table["exptime"] = [exptime] * len(positions)
    
    phot_table["filename"] = [file] * len(positions)
    
    # Cleaning up the column values to be easier to read
    phot_table['xcenter'].info.format = '%.4g'
    phot_table['ycenter'].info.format = '%.4g'
    phot_table['aperture_sum'].info.format = '%.6g'
    phot_table['aperture_mag'].info.format = '%.4g'
    phot_table['sky'].info.format = '%.4g'
    phot_table['peak'].info.format = '%.5g'
    
    # Appending the tables
    aperture_table = vstack([aperture_table, phot_table])
    
    # Updating the position in which to base search for stars since objects move across night sky due to rotation of Earth
    for i in range(len(phot_table["xcenter"])):
        positions[i] = [int(phot_table["xcenter"][i]), int(phot_table["ycenter"][i])]
                
    hdulist.close # Closing the fits file

Writing the QTable out to a .escv file.

In [50]:
aperture_table.write(os.path.join(fits_path,'logfile.ecsv'), overwrite=True)

Viewing the table within the Jupyter Notebook.

In [ ]:
aperture_table # For testing purposes

Below cells are for testing and debugging purposes.

In [ ]:
# For testing purposes
#mags_Apr = np.loadtxt("Dec 2020, Mar 2021, and Apr 2021 Dataset/logfile_April_data_only.txt", usecols = 5, unpack=True)
#flux_Apr = np.loadtxt("Dec 2020, Mar 2021, and Apr 2021 Dataset/logfile_April_data_only.txt", usecols = 6, unpack=True)

#plt.plot(range(len(mags_Apr)), aperture_table["aperture_mag"] - mags_Apr)

In [ ]:
# For testing purposes, used to determine flux and magnitude zeropoints
#from scipy.optimize import curve_fit
#def flux_to_mag(flux, fluxzero, magzero):
#    mag = -2.5*np.log10(flux/fluxzero) + magzero
#    return mag

#parameters, covariance = curve_fit(flux_to_mag, flux_Apr, mags_Apr)
#plt.plot(flux_Apr, mags_Apr, 'o', label='data')
#fit_y = flux_to_mag(flux_Apr, parameters[0], parameters[1])
#plt.plot(flux_Apr, fit_y, '.', label='fit')
#plt.legend()
#print(parameters, covariance)